[Inhaltsverzeichnis](../AP4.ipynb) | [ < zurück ](wohin?)|  [ > next](wohin?)

## 4.1.5 W.E.B. Windkraft

Angaben zu den Daten von Roman Prager (W.E.B.):
Die Windparks Neuhof befinden sich bei Parndorf. Neuhof I ist 2003 in Betrieb gegangen und war damals unser bester Windpark. Neuhof I besteht aus 9 Stück V80 2MW- Anlagen. Neuhof 2 ist 2005 in Betrieb gegangen, das sind 2x V90 2MW- Anlagen. Neuhof III sind V112- Anlagen mit 3MW. Bei diesem Windpark sollte man erkennen können, wie durch den Zubau und die entsprechende Abschattung Neuhof I spezifisch schlechter wird.

Maustrenk ist ein ziemlich freistehender Windpark im Weinviertel, bestehend aus 6x V90 2MW. 2010 ist eine 7. Anlage dazugekommen.

Sternwind I II und III sind drei Ausbaustufen des Windparks Sternwald in Oberösterreich. Diese verhalten sich wesentlich anders als die Windparks in Ostösterreich und könnten daher interessant sein.

In [1]:
# OPTIONAL: Load the "autoreload" extension so that code can change
from pathlib import Path
%load_ext autoreload

In [2]:
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [3]:
%matplotlib inline
import FLUCCOplus.notebooks as config
import FLUCCOplus.web as web
from FLUCCOplus.utils import *

FLUCCOplus.config  loaded!
FLUCCOplus.notebooks loaded!


In [ ]:
path = config.DATA_RAW / Path("web/WEB-Lastgaenge_03032020_15min_kWh.csv")

df_raw = web.read(path)

In [ ]:
df_web = (df_raw
          .pipe(web.clean)
          )
df_web.info()

In [ ]:
df_web.columns

In [ ]:
df_web.head()

In [ ]:
df_p = web.power(df_web)
df_p.head()

In [ ]:
importlib.reload(web)

In [ ]:
df_norm = df_p.copy()
for park in df_norm.columns:
    pow_rat = web.POWER_RATINGS_MW[park]
    df_norm[park] = df_norm[park]/pow_rat

df_norm[df_norm>1] = 1

In [ ]:

df_norm.plot()

In [ ]:
plt.figure(figsize=(20,10))
ax0 = plt.subplot2grid((1, 3), (0, 0), colspan=2)
ax1 = plt.subplot2grid((1, 3), (0, 2), colspan=1)
df_norm.plot(ax=ax0)
df_norm.plot.hist(ax=ax1, bins=50, alpha=0.5)
plt.savefig("normalize.png")

### remove outliers

In [ ]:
df = df_norm.copy()
for col in df.columns:
    q = 0.999999
    print(col, q, df.loc[:,col].quantile(q), df[col].max())

In [ ]:
df.describe()
df.head()

In [ ]:
# not neccessary anymore
#df_rm = web.ts.remove_outliers(df_p, std=3)
#df_rm.info()

In [ ]:
#df_rm.describe()

In [ ]:
#df_norm_rm = df_rm.pipe(web.ts.normalize)
df_norm_rm = df_norm
df_norm_rm.index.year

In [ ]:
plt.figure(figsize=(20,10))
ax0 = plt.subplot2grid((1, 3), (0, 0), colspan=2)
ax1 = plt.subplot2grid((1, 3), (0, 2), colspan=1)
df_norm_rm.plot(ax=ax0)
df_norm_rm[df_norm_rm>0].plot.hist(ax=ax1, bins=50, alpha=0.5)
plt.savefig("img/normed.png", dpi=300)

In [ ]:
config.DATA_PROCESSED

In [ ]:
s = df_norm_rm.to_csv(config.DATA_PROCESSED / "WEB_normalized.csv",
          sep=";",
          decimal=",",
          encoding="cp850")
s

In [ ]:
years = [str(y) for y in df_norm_rm.index.year.unique()]
fig, axes = plt.subplots(len(years), 2, figsize=(15, 8))
#df_yearly[anlagen].plot(kind="bar")
for i, y in enumerate(years):
    for windpark in df_norm_rm.columns:

        df_norm_rm.loc[y+"-1", [windpark]].plot(kind="line", alpha=0.7,xticks=[], sharey=True, ax=axes[i,0])
        df_norm_rm.loc[y+"-2", [windpark]].plot(kind="line", alpha=0.7,xticks=[], sharey=True,ax=axes[i,1])

        axes[i,0].legend("")
        axes[i,1].legend("")
        axes[i,0].set(xlabel="Winter "+y)
        axes[i,1].set(xlabel="Summer "+y)

axes[0,-1].legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
axes[0,1].set(title="Summer month", ylabel="")
axes[0,0].set(title="Winter month", ylabel="")
# fig.tight_layout()
fig.savefig("img/normdetails.png", dpi=300)

In [ ]:
anlagen = ["Neuhof I", "Neuhof III", "Maustrenk",
           "STERN I", "STERN II", "STERN III"]


df_rm = df_web[anlagen] * 4 / 1000

years = ["2012","2013", "2014","2015","2016","2017","2018","2019"]
fig, axes = plt.subplots(len(years), 2, figsize=(15, 7))
#df_yearly[anlagen].plot(kind="bar")
for i, y in enumerate(years):
    for windpark in anlagen:

        df_rm.loc[y+"-1", [windpark]].plot(kind="line", alpha=0.7,xticks=[], sharey=True, ax=axes[i,0])

        df_rm.loc[y+"-7", [windpark]].plot(kind="line", alpha=0.7,xticks=[], sharey=True,ax=axes[i,1])
        axes[i,0].set_ylabel("MW")
        axes[i,1].legend("")
        axes[i,0].set(xlabel="Winter "+y)
        axes[i,1].set(xlabel="Summer "+y)

axes[0,-1].legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
axes[0,1].set(title="Summer month", ylabel="")
axes[0,0].set(title="Winter month", ylabel="MW")
fig.tight_layout()
fig.savefig("examples.png", dpi=300)

### Jahresdauerlinien

In [ ]:
years = [str(y) for y in df_norm.index.year.unique()]
fig, axes = plt.subplots( 1,len(years), figsize=(15, 10),sharey=True)

for i, y in enumerate(years):
    for windpark in df_norm.columns:

        # df_norm.loc[y+"-1-1", [windpark]].plot(kind="line", alpha=0.7,xticks=[], sharey=True, ax=axes[i,0])
        df_norm.loc[y, [windpark]].sort_values(by=windpark,ascending=False)\
            .reset_index(drop=True)\
            .plot(kind="line", alpha=0.7,xticks=[], sharey=True,ax=axes[i])
        # df_rm.loc[y, [windpark]].sort_values(by=windpark,ascending=False)
        # .reset_index(drop=True).plot(ax=ax[i],xticks=[], sharey=True);
        axes[i].legend("")
        axes[i].legend("")
        axes[i].set(xlabel="Winter ")

# ax[-1].legend(loc='center left',bbox_to_anchor=(1.0, 0.5));

In [ ]:
df = df_norm

for park in df.columns:
    df[park] = df[park] * web.POWER_RATINGS_MW[park]

def loadcurve(df):
    years = [str(y) for y in df.index.year.unique()]
    fig, axes = plt.subplots( 1,len(years), figsize=(15, 10),sharey=True)
    for i, y in enumerate(years):
        for windpark in df.columns:

            # df.loc[y+"-1-1", [windpark]].plot(kind="line", alpha=0.7,xticks=[], sharey=True, ax=axes[i,0])
            df.loc[y, [windpark]].sort_values(by=windpark,ascending=False)\
                .reset_index(drop=True)\
                .plot(kind="line", alpha=0.7,xticks=[], sharey=True,ax=axes[i])
            # df_power_MW.loc[y, [windpark]].sort_values(by=windpark,ascending=False)
            # .reset_index(drop=True).plot(ax=ax[i],xticks=[], sharey=True);
            axes[i].legend("")
            axes[i].legend("")
            axes[i].set(xlabel=y)
            axes[0].legend(loc='center left',bbox_to_anchor=(1.0, 0.5));

loadcurve(df)

In [ ]:
fig, axes = plt.subplots( figsize=(15, 10))
for windpark in df.columns:

            # df.loc[y+"-1-1", [windpark]].plot(kind="line", alpha=0.7,xticks=[], sharey=True, ax=axes[i,0])
            df[windpark].sort_values(ascending=False)\
                .reset_index(drop=True)\
                .plot(kind="line", alpha=0.7,xticks=[], sharey=True, ax=axes)


In [ ]:

years = ["2012","2013", "2014","2015","2016","2017","2018","2019"]
fig, ax = plt.subplots(1,len(years), figsize=(15, 6))
for i, y in enumerate(years):
    for windpark in anlagen:
        df_rm.loc[y, [windpark]].sort_values(by=windpark,ascending=False).reset_index(drop=True).plot(ax=ax[i],xticks=[], sharey=True);
        ax[i].legend("")
        ax[i].set(ylabel="Power [MW]", xlabel=y);

ax[-1].legend(loc='center left',bbox_to_anchor=(1.0, 0.5));

fig.savefig("img/JDL.png", dpi=300)


# TODO: Try dask here!!
https://www.youtube.com/watch?v=4WUgRpl-j7Y&list=PLYx7XA2nY5GfdAFycPLBdUDOUtdQIVoMf&index=55&t=0s



also,
# Structural time seris
https://github.com/tensorflow/probability/blob/master/tensorflow_probability/examples/jupyter_notebooks/Structural_Time_Series_Modeling_Case_Studies_Atmospheric_CO2_and_Electricity_Demand.ipynb

https://www.youtube.com/watch?v=v5ijNXvlC5A&list=PLYx7XA2nY5GcDQblpQ_M1V3PQPoLWiDAC&index=41&t=0s

state space models
* filtering
* smoothing?
* predicting

https://en.wikipedia.org/wiki/Kalman_filter

time series feature generation:
* max
* Number peaks

und hier gehts weiter
https://www.youtube.com/watch?v=IWv6EZGa4zo&list=PLYx7XA2nY5GcDQblpQ_M1V3PQPoLWiDAC&index=95&t=0s

# BOSS
Bag of Fast Fourier Symbols
[Source](https://pyts.readthedocs.io/en/latest/auto_examples/transformation/plot_boss.html#sphx-glr-auto-examples-transformation-plot-boss-py)








In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyts.image import RecurrencePlots

# Parameters
n_samples, n_features = 100, 144

# Toy dataset
rng = np.random.RandomState(41)
X = rng.randn(n_samples, n_features)

# Recurrence plot transformation
rp = RecurrencePlots(dimension=1,
                     epsilon='percentage_points',
                     percentage=30)
X_rp = rp.fit_transform(X)

# Show the results for the first time series
plt.figure(figsize=(8, 8))
plt.imshow(X_rp[0], cmap='binary', origin='lower')
plt.show()